# Creating Guardrails for a Retrieval-Augmented Generation (RAG) System

In this cookbook, we'll walk through how to create guardrails for a RAG system using the LastMile AI AutoEval SDK. Guardrails help ensure that your AI system produces outputs that are accurate, relevant, and safe. We'll cover how to:

1. **Set up the RAG System**: Prepare a simple RAG system using LlamaIndex.
2. **Define Evaluation Metrics**: Use LastMile's built-in metrics to assess the system's performance.
3. **Create Custom Guardrails**: Implement custom rules to enforce output quality.
4. **Fine-tune a Guardrail Model**: Improve the guardrails using fine-tuning.
5. **Evaluate and Iterate**: Test the system and iterate on the guardrails.

Let's dive in!

## 1. Setup Environment

First, we need to install the necessary packages and configure our environment.

In [ ]:
!pip install "llama-index>=0.11.0"
!pip install lastmile --upgrade
!pip install pandas

# Configure Pandas display options
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Set Up Your API Key

To interact with the LastMile AI API, you'll need to set your API key as an environment variable. If you haven't already obtained an API key, please visit the [LastMile AI dashboard](https://www.lastmileai.dev/).

This tutorial also uses LLama index with OpenAI to build a basic RAG system, so you'll need to set your OpenAI API key as an environment variable.

In [56]:
import os

# Replace 'YOUR_API_KEY_HERE' with your actual LastMile AI API key
api_token = os.environ.get("LASTMILE_API_TOKEN") or 'YOUR_API_KEY_HERE'
openai_api_key = os.environ.get("OPENAI_API_KEY") or 'YOUR_OPENAI_API_KEY_HERE'

if not api_token or api_token == 'YOUR_API_KEY_HERE':
    print("Error: Please set your API key in the environment variable LASTMILE_API_TOKEN")
else:
    print("✓ API key successfully configured!")

if not openai_api_key or openai_api_key == 'YOUR_OPENAI_API_KEY_HERE':
    print("Warning: Please set your OpenAI API key in the environment variable OPENAI_API_KEY")
else:
    print("✓ OpenAI API key successfully configured!")

✓ API key successfully configured!
✓ OpenAI API key successfully configured!


## 2. Set Up the RAG System

We'll set up a simple RAG system using [LlamaIndex](https://gpt-index.readthedocs.io/). This system will be used to generate responses based on a set of documents.

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the 'data' directory
documents = SimpleDirectoryReader("data/PaulGrahamEssay").load_data()

# Create a vector index from the documents
index = VectorStoreIndex.from_documents(documents)

# Create a query engine for the index
query_engine = index.as_query_engine()
print("✓ RAG system is set up and ready to use!")

✓ RAG system is set up and ready to use!


## 3. Define Evaluation Metrics

To assess the performance of our RAG system and create effective guardrails, we'll use LastMile's built-in metrics. These metrics help evaluate the output's faithfulness, relevance, and toxicity, among others.

In [3]:
from lastmile.lib.auto_eval import AutoEval, BuiltinMetrics, Metric

# Initialize the AutoEval client
eval_client = AutoEval(api_token=api_token)

# Define the metrics we want to use
metrics = [
    BuiltinMetrics.FAITHFULNESS,
    BuiltinMetrics.RELEVANCE,
    BuiltinMetrics.TOXICITY,
    BuiltinMetrics.ANSWER_CORRECTNESS
]
print("✓ Metrics defined!")

✓ Metrics defined!


## 4. Create Custom Guardrails

Now, we'll implement custom guardrails using LastMile's AutoEval Base Metrics to ensure our RAG system produces high-quality outputs. We'll focus on:

- **Input Validation**: Check for inappropriate or toxic inputs and handle them gracefully.
- **Output Verification**: Verify that outputs are faithful to the retrieved context and not hallucinating.
- **Response Correction**: Adjust responses that fail to meet our criteria.

Let's start by defining functions to enforce these guardrails.

### 4.1 Input Validation

We'll check the user's input for toxicity. If the input is too toxic, we'll deny the request politely.

In [8]:
def is_input_toxic(user_input):
    import pandas as pd
    
    # Evaluate the toxicity of the input
    result = eval_client.evaluate_data(
        data=pd.DataFrame({"output": [user_input], "input": [""], "ground_truth": [""]}),
        metrics=[BuiltinMetrics.TOXICITY]
    )
    toxicity_score = result["Toxicity_score"][0]
    print(f"Toxicity score: {toxicity_score}")
    
    # Define a threshold for toxicity (e.g., 0.5)
    toxicity_threshold = 0.5
    return toxicity_score < toxicity_threshold

### 4.2 Output Verification

We'll ensure that the generated output is faithful to the retrieved context to prevent hallucinations.

In [13]:
def is_output_faithful(query, response, context):
    import pandas as pd
    
    # Evaluate the faithfulness of the output against the context
    result = eval_client.evaluate_data(
        data=pd.DataFrame({
            "input": [query],
            "output": [response],
            "ground_truth": [context]
        }),
        metrics=[BuiltinMetrics.FAITHFULNESS]
    )
    faithfulness_score = result["Faithfulness_score"][0]

    # Define a threshold for faithfulness (e.g., 0.7)
    faithfulness_threshold = 0.7
    return faithfulness_score >= faithfulness_threshold

### 4.3 Response Correction

If the output fails our verification, we'll adjust it or provide a fallback response.

In [14]:
def generate_corrected_response(query, context):
    from llama_index.llms.openai import OpenAI
    
    # Use a language model to generate a response strictly based on the context
    llm = OpenAI()
    prompt = f"Answer the question based only on the following context:\n\n{context}\n\nQuestion: {query}\nAnswer:"
    corrected_response = llm.complete(prompt)
    return corrected_response.text.strip()

## 5. Test the System with Guardrails

Now, let's put everything together and test our RAG system with the implemented guardrails.

In [15]:
# Sample user inputs
user_queries = [
    "where did the author grow up",
    "Tell me about the secret operations of the government.",
    "Provide detailed instructions on how to cheat on an exam."
]

for i, query in enumerate(user_queries):
    print(f"\nUser Query #{i+1}: {query}")
    
    # Check if input is toxic
    if not is_input_toxic(query):
        print("🚫 Your input contains inappropriate content. Please rephrase your question. Skipping...")
        continue
    else:
        print("✅ Your input is safe to process. Proceeding...")
    
    
    # Generate response from the RAG system
    llm_response = query_engine.query(query)
    response = llm_response.response
    
    # Retrieve the source context
    if llm_response.source_nodes:
        context = llm_response.source_nodes[0].node.text
    else:
        context = ""
    
    # Verify output faithfulness
    if not is_output_faithful(query, response, context):
        print(f"⚠️ The response may not be accurate: {response=}. Generating a corrected response...")
        response = generate_corrected_response(query, context)
    else:
        print("✅ The response is accurate. Proceeding...")
    
    print(f"Assistant: {response}")


User Query #1: where did the author grow up
Toxicity score: 0.4870980381965637
✅ Your input is safe to process. Proceeding...
⚠️ The response may not be accurate: response='The author grew up in Cambridge, Massachusetts.'. Generating a corrected response...
Assistant: The author grew up in Cambridge, Massachusetts.

User Query #2: Tell me about the secret operations of the government.
Toxicity score: 0.649406373500824
🚫 Your input contains inappropriate content. Please rephrase your question. Skipping...

User Query #3: Provide detailed instructions on how to cheat on an exam.
Toxicity score: 0.018708519637584686
✅ Your input is safe to process. Proceeding...
✅ The response is accurate. Proceeding...
Assistant: I cannot provide assistance or guidance on cheating on an exam. It is important to uphold academic integrity and honesty in all educational endeavors.


## 6. Fine-Tune a Guardrail Model

To enhance the performance of our guardrails, we'll fine-tune a model using LastMile AI's AutoEval to better detect unfaithful responses. We'll use a labeled dataset to train the model.

### 6.1 Prepare the Dataset

We'll create a dataset where each entry consists of an `input`, `output`, and a `label` indicating whether the output is faithful (`1`) or unfaithful (`0`).

This tutorial uses a small sample dataset. For a production-ready guardrail, you should label a much larger dataset for better results.

In [59]:
import pandas as pd

# Create a sample dataset
data = pd.DataFrame({
    "input": [
        "What programming language did Paul Graham first learn?",
        "What programming language did Paul Graham create?",
        "What was Paul Graham's first successful company?", 
        "What company bought Viaweb?",
        "What year was Y Combinator founded?",
        "What was Hacker News originally called?",
        "How much did Y Combinator invest in startups during their first batch?",
        "What was Paul Graham's college major?",
        "What year did Paul Graham move to England?",
        "Who became the second president of Y Combinator?"
    ],
    "output": [
        "Paul Graham first learned Fortran",
        "Paul Graham created Arc and later Bel",
        "Viaweb was Paul Graham's first successful company",
        "Yahoo bought Viaweb",
        "Y Combinator was founded in 2005",
        "Hacker News was originally called Startup News",
        "Y Combinator invested $6,000 per founder",
        "Paul Graham studied Artificial Intelligence in college",
        "Paul Graham moved to England in 2016",
        "Sam Altman became the second president of Y Combinator"
    ],
    "ground_truth": [
        "The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the card reader and press a button to load the program into memory and run it.",
        "In the summer of 2006, Robert and I started working on a new version of Arc. This one was reasonably fast, because it was compiled into Scheme.",
        "We started a new company we called Viaweb, after the fact that our software worked via the web, and we got $10,000 in seed funding from Idelle's husband Julian.",
        "In the summer of 1998 we sold Viaweb to Yahoo for about $50 million.",
        "As Jessica and I were walking home from dinner on March 11, at the corner of Garden and Walker streets, these three threads converged. Screw the VCs who were taking so long to make up their minds. We'd start our own investment firm and actually implement the ideas we'd been talking about.",
        "It was originally meant to be a news aggregator for startup founders and was called Startup News, but after a few months I got tired of reading about nothing but startups.",
        "We invested $6k per founder, which in the typical two-founder case was $12k, in return for 6%.",
        "I studied philosophy in college. But I also started learning artificial intelligence on the side.",
        "In the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice.",
        "When we asked Sam if he wanted to be president of YC, initially he said no. He wanted to start a startup to make nuclear reactors. But I kept at it, and in October 2013 he finally agreed. We decided he'd take over starting with the winter 2014 batch."
    ],
    "label": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 1 for faithful, 0 for unfaithful
})
# Save the dataset to a CSV file
dataset_path = "data/guardrail_training_dataset.csv"
data.to_csv(dataset_path, index=False)
print(f"✓ Successfully saved training dataset to '{dataset_path}' ({len(data)} examples)")

✓ Successfully saved training dataset to 'data/guardrail_training_dataset.csv' (10 examples)


### 6.2 Upload the Dataset

We'll upload the dataset to LastMile AI for use in fine-tuning.

In [40]:
# Upload the dataset
dataset_id = eval_client.upload_dataset(
    file_path=dataset_path,
    name="Guardrail Training Dataset",
    description="Dataset for fine-tuning faithfulness guardrail"
)
print(f"✓ Dataset uploaded with ID: {dataset_id}")

✓ Dataset uploaded with ID: cm3jepf87000bpg011slyfh7j


### 6.3 Define a Custom Prompt Template

We'll create a prompt template that tells the evaluator how to label the outputs.

In [41]:
prompt_template = """
You are an evaluator tasked with assessing whether the assistant's response is faithful to the ground truth and appropriate, based on the following criteria:

Label 1:
- The response is accurate and consistent with the ground truth.
- The response appropriately handles sensitive or disallowed content by politely declining.

Label 0:
- The response contains inaccuracies, hallucinations, or irrelevant information.
- The response provides disallowed content or does not appropriately decline.

Ground Truth:
{ground_truth}

Input:
{input}

Output:
{output}

Label:
"""

### 6.4 Fine-Tune the Model

Fine-tuning a guardrail model using the labeled dataset (this process may take a while to complete)

Note: For automated dataset labeling, refer to the LastMile Getting Started notebook.

In [43]:
print(f"Kicking off fine-tuning job with dataset ... This may take a while to complete.")
model_name = "Faithfulness Guardrail Model" + "3"
fine_tune_job_id = eval_client.fine_tune_model(
    train_dataset_id=dataset_id,
    model_name=model_name,
    selected_columns=["input", "output", "ground_truth"],
    test_dataset_id=dataset_id,
    wait_for_completion=False
)
print(f"✓ Fine-tuning Started with Job ID: {fine_tune_job_id}! Waiting for completion...")
eval_client.wait_for_fine_tune_job(fine_tune_job_id)
print(f"✓ Fine-tuning Job with ID: {fine_tune_job_id} Completed!")
metric = Metric(name=model_name)
print(f"Waiting for fine-tuned model to be available as metric...")
fine_tuned_metric = eval_client.wait_for_metric_online(metric)
print(f"✓ Fine-tuned model is now available as metric!")


Kicking off fine-tuning job with dataset ...
✓ Fine-tuning Started with Job ID: cm3jepn2w000jp6011xu1vmer! Waiting for completion...
✓ Fine-tuning Job with ID: cm3jepn2w000jp6011xu1vmer Completed!
Waiting for fine-tuned model to be available as metric...
✓ Fine-tuned model is now available as metric!


### 6.5 Update the Guardrail Function

Now, we'll update our `is_output_faithful` function to use the fine-tuned model.

In [54]:
# Use the fine-tuned model as a metric
from lastmile.lib.auto_eval import Metric

# Create a Metric instance with the fine-tuned model's name
custom_metric = Metric(name="Faithfulness Guardrail Model")

def is_output_faithful(query, response, context):
    import pandas as pd
    
    # Evaluate the output using the fine-tuned model
    result = eval_client.evaluate_data(
        data=pd.DataFrame({
            "input": [query],
            "output": [response],
            "ground_truth": [context]
        }),
        metrics=[custom_metric]
    )
    score_column = f"{custom_metric.name}_score"
    faithfulness_score = result[score_column][0]
    
    # Use a threshold based on the fine-tuned model's output
    faithfulness_threshold = 0.5
    return faithfulness_score >= faithfulness_threshold

### 6.6 Re-test the System

We'll re-run our test with the updated guardrail function.

In [55]:
# Re-test the system with the updated guardrails
queries = [
    "Who became the second president of Y Combinator?",
    'who is paul graham?'
]
for query in queries:
    print(f"\nUser Query: {query}")
    
    # Generate response from the RAG system
    llm_response = query_engine.query(query)
    response = llm_response.response
    
    # Retrieve the source context
    if llm_response.source_nodes:
        context = llm_response.source_nodes[0].node.text
    else:
        context = ""
    
    # Verify output faithfulness with the fine-tuned model
    if not is_output_faithful(query, response, context):
        print(f'{query=}, {response=}, {context=}')
        print(f"⚠️ The response may not be accurate {response=}. Generating a corrected response...")
        response = generate_corrected_response(query, context)
    else:
        print("✅ The response is accurate. Proceeding...")
    
    print(f"Assistant: {response}")


User Query: Who became the second president of Y Combinator?
✅ The response is accurate. Proceeding...
Assistant: Sam Altman became the second president of Y Combinator.

User Query: who is paul graham?
✅ The response is accurate. Proceeding...
Assistant: Paul Graham is an individual who worked on developing a software called Bel, which is described as a spec rather than an implementation, similar to McCarthy's original Lisp. He also co-founded a company called Viaweb, which later became a model for Y Combinator's funding structure.


## 7. Evaluate and Iterate

Finally, we should evaluate the performance of our guardrails and iterate as needed. This may involve collecting more data, refining the prompt template, or adjusting thresholds.

## Conclusion

In this cookbook, we've seen how to create guardrails for a RAG system using LastMile AI. By implementing input validation, output verification, and fine-tuning a guardrail model, we can enhance the safety and reliability of our AI applications.

Feel free to explore further and customize the guardrails according to your specific needs!